In [1]:
import Chess
using NBInclude

In [2]:
@nbinclude "Values.ipynb"
@nbinclude "Chess.ipynb"

getNextMove (generic function with 4 methods)

In [3]:
if !@isdefined(Transposition)
    const Transposition = Tuple{Int64, Int32}
end

Tuple{Int64, Int32}

In [4]:
function getPositionScoreMemo(extboard::ExtendedBoard, func::Function, depth::Int64, transpositions::Dict{UInt64, Transposition})
    transposition = get(transpositions, extboard.zobrist.hash, nothing)
    if transposition != nothing
        tDepth, tScore = transposition
        if tDepth >= depth
            #global hits += 1
            return tScore
        end
    end
    #global misses += 1
    tScore = func(extboard, depth - 1, transpositions)
    transpositions[extboard.zobrist.hash] = (depth, tScore)
    return tScore
end

getPositionScoreMemo (generic function with 1 method)

In [5]:
function maxMemo(extboard::ExtendedBoard, depth::Int64, transpositions::Dict{UInt64, Transposition})::Int32
    legalMoves = Chess.moves(extboard.board)
    if length(legalMoves) == 0 || extboard.repetionRuleDraw
        return evaluateTerminalPositionScore(extboard.board) - depth
    elseif depth == 0
        return extboard.score
    end
    maxValue = typemin(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        value = getPositionScoreMemo(extboard, minMemo, depth, transpositions)
        undomove!(extboard, undo)
        maxValue = max(value, maxValue)
    end
    Chess.recycle!(legalMoves)
    return maxValue
end

maxMemo (generic function with 1 method)

In [6]:
function minMemo(extboard::ExtendedBoard, depth::Int64, transpositions::Dict{UInt64, Transposition})::Int32
    legalMoves = Chess.moves(extboard.board)
    if length(legalMoves) == 0 || extboard.repetionRuleDraw
        return evaluateTerminalPositionScore(extboard.board) - depth
    elseif depth == 0
        return extboard.score
    end
    minValue = typemax(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        value = getPositionScoreMemo(extboard, maxMemo, depth, transpositions)
        undomove!(extboard, undo)
        minValue = min(value, minValue)
    end
    Chess.recycle!(legalMoves)
    return minValue
end

minMemo (generic function with 1 method)

In [7]:
function minimaxMemoOne(extboard::ExtendedBoard, depth::Int64)::Tuple{Int32, Chess.Move}
    legalMoves = Chess.moves(extboard.board)
    transpositions = Dict{UInt64, Transposition}()
    sideIsWhite = Chess.sidetomove(extboard.board) == Chess.WHITE
    bestMove = legalMoves[1]
    minmaxValue = sideIsWhite ? typemin(Int32) : typemax(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        if sideIsWhite
            value = minMemo(extboard, depth - 1, transpositions)
            if value > minmaxValue
                bestMove = move
                println("$(value) $(bestMove)")
                minmaxValue = value
            end
        else
            value = maxMemo(extboard, depth - 1, transpositions)
            if value < minmaxValue
                bestMove = move
                println("$(value) $(bestMove)")
                minmaxValue = value
            end
        end
        undomove!(extboard, undo)
    end
    return minmaxValue, bestMove
end

minimaxMemoOne (generic function with 1 method)

In [8]:
function minimaxMemoAll(extboard::ExtendedBoard, depth::Int64)::Tuple{Int32, Vector{Chess.Move}}
    legalMoves = Chess.moves(extboard.board)
    transpositions = Dict{UInt64, Transposition}()
    sideIsWhite = Chess.sidetomove(extboard.board) == Chess.WHITE
    scoredMoves = Dict{Int32, Vector{Chess.Move}}()
    minmaxValue = sideIsWhite ? typemin(Int32) : typemax(Int32)
    for move ∈ legalMoves
        undo = domove!(extboard, move)
        if sideIsWhite
            value = minMemo(extboard, depth - 1, transpositions)
            minmaxValue = max(value, minmaxValue)
        else
            value = maxMemo(extboard, depth - 1, transpositions)
            minmaxValue = min(value, minmaxValue)
        end
        movesWithSameScore = get(scoredMoves, value, Chess.Move[])
        push!(movesWithSameScore, move)
        scoredMoves[value] = movesWithSameScore
        println("$(value) $(move)")
        undomove!(extboard, undo)
    end
    return minmaxValue, scoredMoves[minmaxValue]
end

minimaxMemoAll (generic function with 1 method)

In [9]:
minimaxMemoAll(ExtendedBoard(Chess.startboard()), 5)

0 Move(a2a3)
15 Move(b2b3)
-5 Move(c2c3)
40 Move(d2d3)
60 Move(e2e3)
-30 Move(f2f3)
15 Move(g2g3)
0 Move(h2h3)
-5 Move(a2a4)
-25 Move(b2b4)
5 Move(c2c4)
60 Move(d2d4)
70 Move(e2e4)
-10 Move(f2f4)
-15 Move(g2g4)
-5 Move(h2h4)
50 Move(b1a3)
50 Move(b1c3)
50 Move(g1f3)
20 Move(g1h3)


(70, Chess.Move[Move(e2e4)])